In [ ]:
# TODO pulisci questo file

In [ ]:
import os
from uuid import uuid4

os.environ['KERAS_BACKEND'] = "torch"

## Hands on first attempt:


In [ ]:
 # Without any hp tuning we just try and see how it goes.

## Regularization
>We hope to learn vector representations of the most representative aspects for a review dataset.
However, the aspect embedding matrix T may suffer from redundancy problems during training. [...] 
> The regularization term encourages orthogonality among the rows of the aspect embedding matrix T and penalizes redundancy between different aspect vectors
> ~ Ruidan

We use an Orthogonal Regulizer definition of the method can be found here: https://paperswithcode.com/method/orthogonal-regularization. <br/>
For the code we use the default implementation provided by Keras (https://keras.io/api/layers/regularizers/)

## Aspect Embedding Size
The aspect embedding size is what will be inferring aspects. It is closest to representative words (?). <br />
We have to identify 7 actual aspects (luck, bookkeeping, downtime...) but that does not mean our matrix should be limited to rows only! What size to search is a good question and should be studied (Which I may be doing later). 

For the first try we setup the aspect_size:
>The optimal number of rows is problem-dependent, so it’s crucial to: <br/>
> Start with a heuristic: Begin with 2–3x the number of aspects.

For **aspect extraction**, which involves identifying key aspects or topics in text, the best early stopping method depends on your approach:

### 1. Embedding-based Methods (e.g., Clustering Embeddings)
- **Silhouette Score**: Measure the separation and compactness of clusters. Stop when the score stabilizes.
- **Inertia/Distortion**: Track the sum of squared distances within clusters and stop when improvement flattens.
- **Centroid Movement**: Stop when the change in cluster centroids across iterations is minimal.

### 2. Topic Modeling (e.g., LDA)
- **Perplexity**: Monitor the perplexity on a held-out dataset and stop when it stops decreasing significantly.
- **Coherence Score**: Measure the semantic consistency of extracted topics and stop when it stabilizes.

### 3. Autoencoder-based Aspect Extraction
- **Reconstruction Loss**: Stop training when the validation reconstruction error no longer improves.

### 4. Qualitative Evaluation (if feasible)
- Periodically inspect extracted aspects for meaningfulness and diversity to decide on stopping.

For **aspect extraction**, combining an automated metric (like coherence score or silhouette score) with manual inspection often yields the best results.


## Hyperparameters Tuning
To tune our parameters we use a filtered version of the 50k ds. <br>
We filter out rows that can be found on the 200k ds.

In [ ]:
import pandas as pd

# This is based on the idea that our dataset are generated with different seeds else it won't work
large = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")
small = pd.read_csv("../output/dataset/pre-processed/100k.preprocessed.csv")
tuning_set = small[~small["comments"].isin(large["comments"])]

tuning_set.to_csv("../output/dataset/pre-processed/tuning.preprocessed.csv", index=False)

In [ ]:
from core.evaluation import normalize, get_aspect_top_k_words, coherence_per_aspect
from core.hp_tuning import ABAERandomHyperparametersSelectionWrapper
from core.train import AbaeModelManager, AbaeModelConfiguration
from core.dataset import PositiveNegativeCommentGeneratorDataset

from torch.utils.data import DataLoader

hp_wrapper = ABAERandomHyperparametersSelectionWrapper.create()
configurations = 15  # We try 15 different configurations

seen_configurations = set()
scores = list()

corpus_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"

for i in range(configurations):
    uuid = uuid4()
    parameters = next(hp_wrapper)
    while seen_configurations.__contains__(frozenset(parameters.items())):
        print(f"We already worked on configuration: {parameters}")
        parameters = next(hp_wrapper)  # In case we fetch the same config more than once.
    print(f"Working on configuration: {parameters}")
    seen_configurations.add(frozenset(parameters.items()))

    # Train process
    config = AbaeModelConfiguration(corpus_file=corpus_file, model_name=f"tuning_{uuid}", **parameters)
    manager = AbaeModelManager(config)  # todo pass "persist". We dont want to persist these

    # The dataset generation depends on the embedding model
    ds = PositiveNegativeCommentGeneratorDataset(
        vocabulary=manager.embedding_model.vocabulary(),
        csv_dataset_path=config.corpus_file, negative_size=15
    )

    train_dataloader = DataLoader(dataset=ds, batch_size=config.batch_size, shuffle=True)
    iteration_model = manager.prepare_training_model()
    iteration_model.fit(train_dataloader, epochs=config.epochs)

    # Evaluate the model
    # We evaluate on the relative coherence between topics.
    print("Evaluating model")
    word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)

    aspect_embeddings = normalize(iteration_model.get_layer('aspect_emb').W)
    print(f"Word embeddings shape: {word_emb.shape}")
    inv_vocab = manager.embedding_model.model.wv.index_to_key

    aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=50) for a in aspect_embeddings]

    aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]
    coherence, coherence_model = coherence_per_aspect(aspect_words, ds.text_ds, 10)
    scores.append(dict(coherence=coherence_model.get_coherence(), parameters=parameters))

# End done.
print(scores)

## Best found model training:

### 64k - Default

We have too much data for my little PC:

> Sampling: Randomly select a subset of your data that represents the overall distribution of aspects. This will help maintain diversity while reducing the size.
Filtering: Focus on the most informative or high-quality samples. For example, if certain reviews are very short, irrelevant, or don't have useful context for aspect extraction, remove them.
Focus on Diversity: If you reduce the data, make sure the remaining dataset is still representative of the diversity of aspects you're trying to capture.

In [ ]:
# How to Address Issues (If Any):
# Introduce Hard Negatives:
# Instead of randomly selecting negative samples, use hard negatives—examples that are more challenging to distinguish from positive pairs. This keeps the max-margin loss informative and prevents the model from converging too quickly.

# Regularization:
# Apply regularization (e.g., L2 regularization) to prevent overfitting and ensure the model generalizes well.

# Early Stopping:
# If the loss plateaus and aspect quality is satisfactory, consider using early stopping to avoid unnecessary training.

### Hyper-parameters
These should have been discussed earlier. <br>
We could do hyperparmeter optimization, but how do we 'validate' our model? <br>